# Twitter Streaming API Tutorial
<br>
Streaming API allows you to collect live tweets as they're being tweeted

In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import json 
import pandas as pd

In [2]:
consumer_key = '9Jw8uTpVFOxdwII0sT0iSvZb8'
consumer_secret = '6WJaC8MotCcPVb55YetjLVlpmCmRCAdVQ916OhHYvezoWPxFKh'


access_token = '17579008-bf6IuwjRVPnhel9HCDZmhL9UEsGZV9PzqwpYbd2iw'
access_token_secret = 'QKfYn88EcAyIpdsg1gBkwAvEcrMipTFYeB10xMwU3QZ6i'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

To modify this code for your own searches you only need to change the top three lines of code

In [4]:
TRACKING_KEYWORDS = ['#TheBachelorette'] #change keywords for your own searches - can use multiple keywords
OUTPUT_FILE = "streaming.txt" #change to the path of where you want your output file
TWEETS_TO_CAPTURE = 20 #change to the number of tweets you want

class MyStreamListener(tweepy.StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open(OUTPUT_FILE, "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        
        # Stops streaming when it reaches the limit
        if self.num_tweets <= TWEETS_TO_CAPTURE:
            if self.num_tweets % 100 == 0: # just to see some progress...
                print('Numer of tweets captured so far: {}'.format(self.num_tweets))
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [6]:
# Initialize Stream listener
l = MyStreamListener()

# Create you Stream object with authentication
stream = tweepy.Stream(auth, l)

# Filter Twitter Streams to capture data by the keywords:
stream.filter(track=TRACKING_KEYWORDS)

In [7]:
tweets_data_path = "streaming.txt"  
tweets_data = []  
tweets_file = open(tweets_data_path, "r")  
for line in tweets_file:  
    try:  
        tweet = json.loads(line)  
        tweets_data.append(tweet)  
    except:  
        continue
tweets = pd.DataFrame()
tweets['tweetid'] = list(map(lambda tweet: tweet['id'], tweets_data))
tweets['userid'] = list(map(lambda tweet: tweet['user']['id'], tweets_data))
tweets['user_display_name'] = list(map(lambda tweet: tweet['user']['name'], tweets_data))
tweets['user_screen_name'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['user_reported_location'] = list(map(lambda tweet: tweet['user']['location'], tweets_data))
tweets['user_profile_description'] = list(map(lambda tweet: tweet['user']['description'], tweets_data))
tweets['followers_count'] = list(map(lambda tweet: tweet['user']['followers_count'], tweets_data))
tweets['following_count'] = list(map(lambda tweet: tweet['user']['friends_count'], tweets_data))
tweets['num_tweets'] = list(map(lambda tweet: tweet['user']['statuses_count'], tweets_data))
tweets['account_creation_date'] = list(map(lambda tweet: tweet['user']['created_at'], tweets_data))
tweets['verified'] = list(map(lambda tweet: tweet['user']['verified'], tweets_data))

#tweet entities
tweets['tweet_text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['tweet_time'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['tweet_client_name'] = list(map(lambda tweet: tweet['source'], tweets_data))
tweets['truncated'] = list(map(lambda tweet: tweet['truncated'], tweets_data))
tweets['in_reply_to_userid'] = list(map(lambda tweet: tweet['in_reply_to_user_id'], tweets_data))
tweets['in_reply_to_tweetid'] = list(map(lambda tweet: tweet['in_reply_to_status_id'], tweets_data))
#tweets['quoted_tweet_tweetid'] = list(map(lambda tweet: tweet['quoted_status_id'], tweets_data))
#tweets['is_retweet'] = list(map(lambda tweet: tweet['is_retweet_status'], tweets_data))
#tweets['retweet_tweet'] = list(map(lambda tweet: tweet['retweeted_status'], tweets_data))
tweets['quote_count'] = list(map(lambda tweet: tweet['quote_count'], tweets_data))
tweets['reply_count'] = list(map(lambda tweet: tweet['reply_count'], tweets_data))
tweets['like_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))
tweets['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
tweets['entities'] = list(map(lambda tweet: tweet['entities'], tweets_data))
tweets["hashtags"] = list(map(lambda tweet: tweet['entities']['hashtags'], tweets_data))
tweets['tweet_client_name'] = tweets['tweet_client_name'].apply(lambda x: x[x.find('>')+len('>'):x.rfind('<')])


tweets.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,followers_count,following_count,num_tweets,account_creation_date,...,tweet_client_name,truncated,in_reply_to_userid,in_reply_to_tweetid,quote_count,reply_count,like_count,retweet_count,entities,hashtags
0,1402300221833502727,1361520011924406274,The Bachelor Française,TheFrancaise,"New York, NY",Will you accept this rosé? Sharing the most de...,18,98,99,Tue Feb 16 03:37:33 +0000 2021,...,Twitter for iPhone,False,NaN,NaN,0,0,0,0,"{'hashtags': [{'text': 'TheBachelorette', 'ind...","[{'text': 'TheBachelorette', 'indices': [79, 9..."
1,1402300318822535179,1088308743681454080,Tea and Roses ♥️☕️🌹,myteaandroses,None,🌹BACHELOR NATION🍷This is a place for all thing...,852,337,2310,Thu Jan 24 05:33:19 +0000 2019,...,Twitter for iPhone,False,NaN,NaN,0,0,0,0,"{'hashtags': [{'text': 'TheBachelorette', 'ind...","[{'text': 'TheBachelorette', 'indices': [39, 5..."
2,1402300331585945610,1014538939309404160,M. J. Designs,ForYourCart,U.S.A.,Offering many unique quality items to our cust...,9,21,1119,Wed Jul 04 15:58:27 +0000 2018,...,Twitter Web App,True,NaN,NaN,0,0,0,0,"{'hashtags': [{'text': 'rose', 'indices': [4, ...","[{'text': 'rose', 'indices': [4, 9]}, {'text':..."
3,1402300367841337347,150126088,taylor,tayloryley,"Mesa, AZ",mom • wife • travel • repeat,126,171,8867,Mon May 31 04:27:51 +0000 2010,...,Twitter for iPhone,False,NaN,NaN,0,0,0,0,"{'hashtags': [{'text': 'TheBachelorette', 'ind...","[{'text': 'TheBachelorette', 'indices': [42, 5..."
4,1402300438423248902,1148743297579606022,jack mcphee superiority,greysbachelors,None,a good tv show and a good book is all I need i...,69,30,8714,Tue Jul 09 23:58:39 +0000 2019,...,Twitter for iPhone,False,NaN,NaN,0,0,0,0,"{'hashtags': [{'text': 'TheBachelorette', 'ind...","[{'text': 'TheBachelorette', 'indices': [78, 9..."
